# Solar Energy Consumption and Production Analysis
This notebook analyzes solar energy consumption and production. It calculates various metrics, including PV production, unmet demand, and performance indicators, under different conditions (e.g., number of panels and batteries).


### Solar + Wind Energy Consumption and Production Analysis for Grenoble City (For one Week)

In [ ]:
# --------------------------------
# Required Library Installation
# --------------------------------

import sys
import subprocess

# List of required Python libraries for the project
required_libraries = [
    'pandas',            # Data processing and manipulation
    'numpy',             # Numerical calculations
    'plotly',            # Interactive visualizations
    'ipywidgets',        # Interactive widgets for user input
    'matplotlib',        # Plotting capabilities 
]

# Function to check and install missing libraries
def install_missing_libraries(libraries):
    for lib in libraries:
        try:
            __import__(lib)
        except ImportError:
            print(f"Installing missing library: {lib}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
        else:
            print(f"✔ {lib} is already installed.")

# Run the function to install missing libraries
install_missing_libraries(required_libraries)


In [1]:
# main.ipynb
# ============================================
# Solar + Wind Energy Consumption and Production Analysis for Grenoble City
# ============================================
#
# Author: Arbaz KHALID & Tshephang Komana
# Date: 2024-12-30
#
# Description:
# This notebook performs a comprehensive analysis of solar and wind energy
# consumption and production for the city of Grenoble for the year 2019.
# Users can adjust the number of PV panels, wind turbines, and batteries, as well as
# select specific weeks to visualize metrics such as unmet demand, excess energy,
# and battery SOC evolution.
#
# Note: The consumption data are stored in MWh. In the load function we convert these
# values to kWh for internal computations. However, the updated plot and summary will
# display energy values in MWh.

# -------------------------------
# Cell 1: Environment Setup
# -------------------------------
import os
import sys
import logging
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import the custom analysis script
import solar_analysis_script as sas
from solar_analysis_script import update_weekly_graph  # The update function

# (Optional) Reload the module to pick up any changes during development
import importlib
importlib.reload(sas)

# -------------------------------
# Cell 2: Configuration and File Paths
# -------------------------------
# File paths for the data files
CONSUMPTION_FILE = 'Grenoble_city_load_1.csv'
PV_FILE = 'One_PV_Panel_Production.csv'
WIND_FILE = 'wind_production_for_one_turbine.csv'

# Date range for the year 2019
CONSUMPTION_START_DATE = '2019-01-01 00:00:00'
CONSUMPTION_END_DATE   = '2019-12-31 23:59:59'

# Initial parameter values (from the analysis script)
INITIAL_NUM_PANELS   = sas.INITIAL_NUM_PANELS
INITIAL_NUM_TURBINES = sas.INITIAL_NUM_TURBINES
INITIAL_NUM_BATTERIES = 500  # Adjust if needed
battery_capacity     = sas.battery_capacity_per_unit  # kWh per battery

# -------------------------------
# Cell 3: Load Raw Data
# -------------------------------
print("Loading and preprocessing consumption data...")
try:
    # Note: Although the CSV values are in MWh, the function converts them to kWh.
    # In this updated version the load function is modified to *filter* (not resample)
    # so that only rows at the full hour are kept.
    city_consumption = sas.load_city_consumption(
        consumption_file_path=CONSUMPTION_FILE,
        consumption_sep=';',
        consumption_date_col='Date',
        consumption_time_col='Heures',
        consumption_value_col='Consommation(MWh)'
    )
    print("Consumption Data (head):")
    print(city_consumption.head())
except Exception as err:
    print(f"Failed to load consumption data: {err}")
    raise

print("\nLoading and preprocessing PV production data...")
try:
    city_pv_production = sas.load_city_pv_production(
        pv_file_path=PV_FILE,
        consumption_start_date=CONSUMPTION_START_DATE,
        consumption_end_date=CONSUMPTION_END_DATE,
        sep=',',
        time_start_col='time',
        time_end_col='local_time',
        electricity_col='electricity'
    )
    print("PV Production Data (head):")
    print(city_pv_production.head())
except Exception as err:
    print(f"Failed to load PV production data: {err}")
    raise

print("\nLoading and preprocessing Wind production data...")
try:
    city_wind_production = sas.load_city_wind_production(
        wind_file_path=WIND_FILE,
        consumption_start_date=CONSUMPTION_START_DATE,
        consumption_end_date=CONSUMPTION_END_DATE,
        sep=',',
        time_start_col='time',
        time_end_col='local_time',
        electricity_col='wind_electricity'
    )
    print("Wind Production Data (head):")
    print(city_wind_production.head())
except Exception as err:
    print(f"Failed to load wind production data: {err}")
    raise

# -------------------------------
# Cell 4: Define System Parameters
# -------------------------------
battery_capacity_per_unit = sas.BATTERY_CAPACITY_PER_UNIT  # kWh per battery
converter_efficiency     = sas.CONVERTER_EFFICIENCY         # e.g. 0.9 (90%)
initial_soc              = sas.INITIAL_SOC                  # e.g. 0 or 50

# -------------------------------
# Cell 5: Create Interactive Widgets and Output Container
# -------------------------------
# Add custom CSS to prevent label truncation
from IPython.display import HTML
HTML('''
<style>
    .widget-label {
        min-width: 200px !important;
        white-space: nowrap !important;
        overflow: visible !important;
        text-overflow: unset !important;
    }
</style>
''')

# Custom layout for better visibility
slider_style = {'description_width': '150px'}
slider_layout = widgets.Layout(
    width='100%',
    min_width='400px',
    margin='0 0 0 0px'
)

# Define interactive controls with full-width labels
num_panels_slider = widgets.IntSlider(
    value=INITIAL_NUM_PANELS,
    min=0,
    max=1e7,
    step=1e3,
    description='Number of PV Panels:',
    continuous_update=False,
    style=slider_style,
    layout=slider_layout
)

num_turbines_slider = widgets.IntSlider(
    value=INITIAL_NUM_TURBINES,
    min=0,
    max=1000,
    step=10,
    description='Number of Wind Turbines:',
    continuous_update=False,
    style=slider_style,
    layout=slider_layout
)

num_batteries_slider = widgets.IntSlider(
    value=INITIAL_NUM_BATTERIES,
    min=0,
    max=1e6,
    step=100,
    description='Number of Battery Units:',
    continuous_update=False,
    style=slider_style,
    layout=slider_layout
)

# Dropdowns with improved spacing
month_dropdown = widgets.Dropdown(
    options=[(pd.to_datetime(str(month), format='%m').strftime('%B'), month)
             for month in range(1, 13)],
    value=1,
    description='Month:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

week_dropdown = widgets.Dropdown(
    options=[], 
    value=None, 
    description='Week Start Date:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='500px')
)

# Create one output widget that will display both the figure and the metrics
main_output = widgets.Output()

# Create container with proper spacing
controls_box = widgets.VBox([
    widgets.VBox([
        num_panels_slider,
        num_turbines_slider,
        num_batteries_slider
    ], layout=widgets.Layout(
        justify_content='space-around',
        width='100%',
        padding='1px 2px'
    )),
    widgets.HBox([month_dropdown, week_dropdown], 
                 layout=widgets.Layout(
                     justify_content='space-around',
                     width='100%',
                     padding='1px 2px'
                 ))
], layout=widgets.Layout(
    width='50%',
    padding='10px 0',
    border='1px solid #e0e0e0',
    margin='0px 0'
))

# Display the controls and the main output area
display(controls_box, main_output)

# -------------------------------
# Cell 6: Populate Week Dropdown Based on Selected Month
# -------------------------------
def populate_week_dropdown(change):
    selected_month = change['new']
    # Get all unique normalized dates in the selected month from consumption data
    dates_in_month = city_consumption[city_consumption.index.month == selected_month].index.normalize().unique()
    dates_in_month = pd.to_datetime(dates_in_month).sort_values()

    # Generate list of week start dates (ensuring a complete week of data is available)
    week_start_dates = []
    seen_dates = set()
    for date in dates_in_month:
        date = pd.Timestamp(date)
        # Ensure the week starts on Monday
        week_start = date - pd.Timedelta(days=date.weekday())
        if week_start < pd.Timestamp(CONSUMPTION_START_DATE):
            continue
        if week_start in seen_dates:
            continue
        # Define a week as 7 consecutive days starting from week_start
        week = [week_start + pd.Timedelta(days=i) for i in range(7)]
        if all(d in dates_in_month for d in week):
            week_start_dates.append(week_start.strftime('%Y-%m-%d'))
            seen_dates.update(week)
    
    if week_start_dates:
        week_dropdown.options = week_start_dates
        week_dropdown.value = week_start_dates[0]
        print(f"Available weeks in {pd.to_datetime(selected_month, format='%m').strftime('%B')}:")
        for ws in week_start_dates:
            print(f"Week starting on {ws}")
    else:
        week_dropdown.options = []
        week_dropdown.value = None
        print(f"No complete weeks available for {pd.to_datetime(selected_month, format='%m').strftime('%B')}.")

# Set observer for month change
month_dropdown.observe(populate_week_dropdown, names='value')
# Call once initially to populate the week dropdown
populate_week_dropdown({'new': month_dropdown.value})

# -------------------------------
# Cell 7: Define and Attach the Graph Update Function
# -------------------------------
def update_graph(*args):
    """
    This function retrieves the current widget values,
    clears the main_output area, and then updates the plot and summary metrics.
    """
    with main_output:
        clear_output(wait=True)
        # Get current parameter values from the widgets
        num_panels = num_panels_slider.value
        num_turbines = num_turbines_slider.value
        num_batteries = num_batteries_slider.value
        selected_week_start = week_dropdown.value

        if not selected_week_start:
            print("[Error] No week selected. Please select a valid week.")
            return

        # Call the update_weekly_graph function from the analysis script
        try:
            result = update_weekly_graph(
                city_consumption=city_consumption,
                city_pv_production=city_pv_production,
                city_wind_production=city_wind_production,
                soc_tracker=sas.soc_tracker,
                num_panels=num_panels,
                num_turbines=num_turbines,
                num_batteries=num_batteries,
                selected_week_start=selected_week_start,
                battery_capacity_per_unit=battery_capacity,
                converter_efficiency=converter_efficiency,
                initial_soc=initial_soc
            )
            
            # Get the figure from the result
            fig = result["figure"]
            
            # Update layout (x-axis now shows the real hourly timestamps with date labels only for each day)
            fig.update_layout(
                width=1000,
                height=400,
                margin=dict(l=50, r=100, t=80, b=120),
                autosize=True,
                title_x=0.5
            )
        except Exception as err:
            print(f"[Error] {err}")
            return

        # Display the updated figure and the summary metrics
        display(fig)

        # Create metrics display
        from IPython.display import HTML
        display(HTML(
            f"""
            <div style='margin:0px 0; padding:0px; background:#f8f9fa; border-radius:2px;'>
                <h4 style='color:#2c3e50; margin-bottom:15px;'>
                    ⚡ Weekly Energy Summary ({selected_week_start})
                </h4>
                <table style='width:50%; border-collapse:collapse;'>
                    <thead>
                        <tr style='border-bottom:2px solid #3498db;'>
                            <th style='padding:0px; text-align:left;'>Metric</th>
                            <th style='padding:0px; text-align:right;'>Value</th>
                        </tr>
                    </thead>
                    <tbody>
                        {"".join([
                            f"<tr style='border-bottom:1px solid #dee2e6;'>"
                            f"<td style='padding:0px;text-align:left;'>{key}</td>"
                            f"<td style='padding:0px; text-align:right;'>"
                            f"{round(value, 2) if isinstance(value, float) else value} MWh</td>"
                            f"</tr>"
                            for key, value in result["metrics"].items()
                        ])}
                    </tbody>
                </table>
            </div>
            """
        ))

# Attach the update_graph function to changes in all the controls
num_panels_slider.observe(update_graph, names='value')
num_turbines_slider.observe(update_graph, names='value')
num_batteries_slider.observe(update_graph, names='value')
week_dropdown.observe(update_graph, names='value')
month_dropdown.observe(lambda change: update_graph(), names='value')

# Initial call to display the graph
update_graph()


Loading and preprocessing consumption data...
Consumption Data (head):
Datetime
2019-01-01 00:00:00    580000
2019-01-01 01:00:00    558000
2019-01-01 02:00:00    539000
2019-01-01 03:00:00    543000
2019-01-01 04:00:00    531000
Name: Consumption_kWh, dtype: int64

Loading and preprocessing PV production data...
PV Production Data (head):
Time_Start
2019-01-01 00:00:00    0.0
2019-01-01 01:00:00    0.0
2019-01-01 02:00:00    0.0
2019-01-01 03:00:00    0.0
2019-01-01 04:00:00    0.0
Name: PV_Production_kWh, dtype: float64

Loading and preprocessing Wind production data...
Wind Production Data (head):
Time_Start
2019-01-01 00:00:00     923.138
2019-01-01 01:00:00     942.556
2019-01-01 02:00:00    1064.198
2019-01-01 03:00:00    1328.341
2019-01-01 04:00:00    1625.839
Name: Wind_Production_kWh, dtype: float64


Output()

Available weeks in January:
Week starting on 2019-01-07
Week starting on 2019-01-14
Week starting on 2019-01-21
